# Импорт библиотек и файлов

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Предобработка данных:
## - Удаление столбцов
## - Приведение текстовых данных в числовую форму (словарь)
## - Замена NaN значений
## - Приведение к одному типу объектов (int64)

In [ ]:
# удаляем нерелевантный столбец
train = train.drop('Utilities',axis=1)
test = test.drop('Utilities',axis=1)

In [ ]:
# деление обучающей и тестовой выборки на текстовые и числовые данные
train1 = pd.DataFrame()
train2 = pd.DataFrame()
l = []
r = []
for i in train:
    if(np.dtype(train[i])=='object'):
        l.append(i)
        train1[i]=train[i]
    else:
        train2[i]=train[i]
    if(np.dtype(train[i])=='int64'):
        r.append(i)

In [ ]:
test1=pd.DataFrame()
test2=pd.DataFrame()
l=[]
r=[]
for i in test:
    if(np.dtype(test[i])=='object'):
        l.append(i)
        test1[i]=test[i]
    else:
        test2[i]=test[i]
    if(np.dtype(test[i])=='int64'):
        r.append(i)

In [ ]:
# заменяем текстовые данные числовыми при помощи словаря
import ast
import json
file = open("dict_train.txt", "r")

contents = file.read()
tik = json.loads(contents)

In [ ]:
for column in tik:
    train1[column] = train1[column].map(tik[column])
for column in tik:
    test1[column] = test1[column].map(tik[column])

In [ ]:
#заполняем пустые ячейки
train1 = train1.replace(np.nan, 0)
test1 = test1.replace(np.nan, 0)
train2 = train2.replace(np.nan, 0)
test2 = test2.replace(np.nan, 0)

In [ ]:
#соединяем обратно и унифицируем тип объекта
train = train1.join(train2).astype('int64')
test = test1.join(test2).astype('int64')

In [ ]:
# финальная подготовка: удаляем из признаков столбец с ценами, от целевой переменной с ценами берем логарифм
x = train.drop(['SalePrice'], axis=1)
y = np.log(train['SalePrice'])
z = test

# <font color='green'> CatBoost <font>

In [ ]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor(iterations=3000)
cat.fit(x, y)
pred_cat = cat.predict(z)
final_cat=pd.DataFrame()
final_cat['SalePrice'] = pred_cat

In [ ]:
final=pd.DataFrame()
final['Id'] = range(1461,2920)

In [ ]:
#собираем в одну таблицу
final=final.join(final_cat)

In [ ]:
#антилогарифм
final['SalePrice'] = np.exp(final['SalePrice'])

In [ ]:
final.to_csv("final_test_catboost.csv",index=False)

##### Метрика = 0.12540

# <font color = 'green'> XGBoost<font>

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(base_score=0.4, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4603, gamma=0.05,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1.7817, monotone_constraints='()',
             n_estimators=2200, n_jobs=4, nthread=-1, num_parallel_tree=1,
             random_state=7, reg_alpha=0.464, reg_lambda=0.8571,
             scale_pos_weight=1, subsample=0.5213,silent = True,tree_method='exact',
             validate_parameters=1, verbosity=0)
xgb.fit(x, y)
xgb_pred = xgb.predict(z)

final_xgb=pd.DataFrame()
final_xgb['SalePrice'] = xgb_pred

In [ ]:
final=pd.DataFrame()
final['Id'] = range(1461,2920)

In [ ]:
#собираем в одну таблицу
final=final.join(final_xgb)

In [ ]:
#антилогарифм
final['SalePrice'] = np.exp(final['SalePrice'])

In [ ]:
final.to_csv("final_test_Xgboost.csv",index=False)

##### Метрика = 0.12472

# <font color = 'blue'> Плюшки (не использовались) <font>
## Большой словарь

In [ ]:
tok = {'RL':0, 'RM':1, 'C (all)':2, 'FV':3, 'RH':4,
       'Pave':1, 'Grvl':2,
       'Nothing':0,
       'Reg':0,'IR1':1,'IR2':2,'IR3':3,
       'Lvl':0,'Bnk':1,'Low':2,'HLS':3,
       'AllPub':0,'NoSeWa':1,
       'Inside':0,'FR2':1,'Corner':2,'CulDSac':3,'FR3':4,
       'Gtl':0,'Mod':1,'Sev':2,
       'CollgCr':0,'Veenker':1,'Crawfor':2,'NoRidge':3,'Mitchel':4,'Somerst':5,'NWAmes':6,'OldTown':7,'BrkSide':8,'Sawyer':9,
       'NridgHt':10,'NAmes':11,'SawyerW':12,'IDOTRR':13,'MeadowV':14,'Edwards':15,'Timber':16,'Gilbert':17,'StoneBr':18,
       'ClearCr':19,'NPkVill':20,'Blmngtn':21,'BrDale':22,'SWISU':23,'Blueste':24,
       'Norm':0, 'Feedr':1, 'PosN':2, 'Artery':3, 'RRAe':4, 'RRNn':5, 'RRAn':6, 'PosA':7, 'RRNe':8,
       '1Fam':0, '2fmCon':1, 'Duplex':2, 'TwnhsE':3, 'Twnhs':4,
       '2Story':0, '1Story':1, '1.5Fin':2, '1.5Unf':3, 'SFoyer':4, 'SLvl':5, '2.5Unf':6, '2.5Fin':7,
       'Gable':0, 'Hip':1, 'Gambrel':2, 'Mansard':3, 'Flat':4, 'Shed':5,
       'CompShg':0,'WdShngl':1,'Metal':2,'WdShake':3,'Membran':4,'Tar&Grv':5,'Roll':6,'ClyTile':7,
       'VinylSd':0,'MetalSd':1,'Wd Sdng':2, 'HdBoard':3, 'BrkFace':4, 'WdShing':5, 'CemntBd':6,'CmentBd':6,
       'Plywood':7, 'AsbShng':8,'Stucco':9, 'BrkComm':10, 'AsphShn':11, 'Stone':12, 'ImStucc':13, 'CBlock':14, 'Wd Shng':15,
       'Brk Cmn':16, 'Other':17, 
       'None':0,'BrkCmn':1,'0':5,
       'Gd':0, 'TA':1, 'Ex':2, 'Fa':3, 'Po':4,
       'PConc':0,'BrkTil':1,'Wood':2,'Slab':3,
       'No':0,'Mn':1,'Av':2,
       'GLQ':0, 'ALQ':1, 'Unf':2, 'Rec':3, 'BLQ':4, 'LwQ':6,
       'GasA':0,'GasW':1,'Grav':2,'Wall':3,'OthW':4,'Floor':5,
       'Y':0,'N':1,'P':2,
       'SBrkr':0, 'FuseF':1, 'FuseA':2, 'FuseP':3, 'Mix':4,
       'Typ':0, 'Min1':3, 'Maj1':4, 'Min2':5, 'Maj2':6,
       'Attchd':0, 'Detchd':1, 'BuiltIn':2, 'CarPort':3, 'Basment':4, '2Types':6,
       'RFn':0, 'Fin':1,
       'MnPrv':0,'GdWo':1,'GdPrv':2,'MnWw':3,
       'Gar2':0,'Othr':1,'TenC':2,
       'WD':0, 'New':1, 'COD':2, 'ConLD':3, 'ConLI':4, 'CWD':5, 'ConLw':6, 'Con':7, 'Oth':8,
       'Normal':0, 'Abnorml':1, 'Partial':2, 'AdjLand':3, 'Alloca':4, 'Family':5
       
      }

train1 = train1.replace(tok)
test1 = test1.replace(tok)

train = train1.join(train2)
test = test1.join(test2)

## Удаление выбросов

In [ ]:
from scipy.stats import zscore
from scipy import stats
z_scores = stats.zscore(train['SalePrice'])

In [ ]:
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3)
new_train = train[filtered_entries]

In [ ]:
x = new_train.drop(['SalePrice'], axis=1)
y = new_train['SalePrice']
z = test

## Удаление столбцов, где NA >1000

In [ ]:
null=train.loc[:,train.isnull().sum()>1100]
train.drop(null,inplace=True,axis=1)
null=test.loc[:,test.isnull().sum()>1100]
test.drop(null,inplace=True,axis=1)

####  Если нужно удалить вложенный словарь

In [ ]:
del tik['Alley']
del tik ['PoolQC']
del tik ['Fence']
del tik ['MiscFeature']